# Sensors Analytics REST API

## Deploy the API
- Go to the *Machine* tab, then set *Incoming connections* to **ON**. The API will be accessible through the indicated tunnelling link.  
- Run the notebook.



## Import the required modules

In [1]:
import cherrypy
import json
import redis
import datetime

## Connect to the Redis Database

In [ ]:
# Redis connection setup
REDIS_HOST = 'redis-15780.c250.eu-central-1-1.ec2.redns.redis-cloud.com'
REDIS_PORT = 15780
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'DIZGajssLlzVl2kN1btFK0LKgBg36OLS'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)

is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

## Status Endpoint Class

In [3]:
class Status(object):
    exposed = True

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

## Sensors Endpoint Class

In [4]:
class Sensors(object):
    exposed = True

    def GET(self, *path, **query):
        # print(query)
        min_t_samples = int(query.get('min_t_samples', 0))
        min_h_samples = int(query.get('min_h_samples', 0))
        sensors = []
        keys = redis_client.keys('0x*:temperature')

        count = 0
        for key in keys:
            key = key.decode()
            mac_address = key.split(':')[0]

            t_info = redis_client.ts().info(f'{mac_address}:temperature')
            t_samples = t_info.total_samples
            t_retention = t_info.retention_msecs
            h_info = redis_client.ts().info(f'{mac_address}:humidity')
            h_samples = h_info.total_samples
            h_retention = h_info.retention_msecs
            
            if t_samples >= min_t_samples and h_samples >= min_h_samples:
                sensors.append(
                    {
                        "mac_address": mac_address,
                        "t_samples": t_samples,
                        "t_retention": t_retention,
                        "h_samples": h_samples,
                        "h_retention": h_retention,
                    }
                )
                count += 1

        response_dict = {
            "sensors": sensors,
            "count": count,
        }

        response = json.dumps(response_dict)

        return response

    def POST(self, *path, **query):
        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        mac_address = body_dict.get('mac_address', None)

        if mac_address is None:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request body.')

        try:
            redis_client.ts().create(f'{mac_address}:temperature', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        try:
            redis_client.ts().create(f'{mac_address}:humidity', retention_msecs=24*60*60*1000)
        except redis.ResponseError:
            raise cherrypy.HTTPError(409, 'Sensor already exists.')

        return

## Sensor Endpoint Class

In [5]:
class Sensor(object):
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        try:
            t_info = redis_client.ts().info(f'{mac_address}:temperature')
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')
        
        h_info = redis_client.ts().info(f'{mac_address}:humidity')

        response_dict = {
            "mac_address": mac_address,
            "t_samples": t_info.total_samples,
            "t_retention": t_info.retention_msecs,
            "h_samples": h_info.total_samples,
            "h_retention": h_info.retention_msecs,
        }

        response = json.dumps(response_dict)

        return response

    def PUT(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        try:
            t_info = redis_client.ts().info(f'{mac_address}:temperature')
        except redis.ResponseError:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        body = cherrypy.request.body.read()
        # print(body)
        body_dict = json.loads(body.decode())
        # print(body_dict)

        t_retention = body_dict.get('t_retention', None)

        if t_retention is None:
            raise cherrypy.HTTPError(400, 'Missing temperature retention period in the request body.')

        h_retention = body_dict.get('h_retention', None)

        if h_retention is None:
            raise cherrypy.HTTPError(400, 'Missing humidity retention period in the request body.')

        redis_client.ts().alter(f'{mac_address}:temperature', retention_msecs=t_retention)
        redis_client.ts().alter(f'{mac_address}:humidity', retention_msecs=h_retention)

        return
    
    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')
        
        mac_address = path[0]
        found = 0
        found += redis_client.delete(f'{mac_address}:temperature')
        found += redis_client.delete(f'{mac_address}:humidity')

        if found == 0:
            raise cherrypy.HTTPError(404, 'MAC address not found in the database.')

        return

## HistoricalData Endpoint Class

In [6]:
class HistoricalData(object):
    exposed = True

    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Missing MAC address in the request parameters.')

        mac_address = path[0]

        # Validate query parameters
        start_date = query.get('start_date')
        end_date = query.get('end_date')


        if not start_date:
            raise cherrypy.HTTPError(400, "Missing start date in the request parameters.")
        if not end_date:
            raise cherrypy.HTTPError(400, "Missing end date in the request parameters.")

        try:
            # Convert ISO 8601 string to a datetime object
            iso_start_date = datetime.datetime.fromisoformat(start_date)
            
            # Convert datetime object to a timestamp (in seconds)
            timestamp_start_in_seconds = int(iso_start_date.timestamp())
            
            # Convert the timestamp to milliseconds
            timestamp_start_in_milliseconds = timestamp_start_in_seconds * 1000
        except ValueError:
            raise cherrypy.HTTPError(400, "Wrong format for start date in the request parameters.")

        try:
            # Convert ISO 8601 string to a datetime object
            iso_end_date = datetime.datetime.fromisoformat(end_date)

            # Convert datetime object to a timestamp (in seconds)
            timestamp_end_in_seconds = int(iso_end_date.timestamp())
            
            # Convert the timestamp to milliseconds
            timestamp_end_in_milliseconds = timestamp_end_in_seconds * 1000
        except ValueError:
            raise cherrypy.HTTPError(400, "Wrong format for end date in the request parameters.")
        
        if timestamp_end_in_milliseconds <= timestamp_start_in_milliseconds:
            raise cherrypy.HTTPError(400, "End date smaller or equal than start date")
            
        # Check if the MAC address exists in Redis
        temperature_key = f"{mac_address}:temperature"
        humidity_key = f"{mac_address}:humidity"

        if not redis_client.exists(temperature_key) or not redis_client.exists(humidity_key):
            raise cherrypy.HTTPError(404, "MAC address not found in the database.")

        # Fetch temperature and humidity data within the specified range
        temperature_data = redis_client.ts().range(
            temperature_key, timestamp_start_in_milliseconds, timestamp_end_in_milliseconds
        )
        humidity_data = redis_client.ts().range(
            humidity_key, timestamp_start_in_milliseconds, timestamp_end_in_milliseconds
        )

        # Extract timestamps, temperatures, and humidity values
        timestamps = [item[0] for item in temperature_data]
        temperatures = [item[1] for item in temperature_data]
        humidities = [item[1] for item in humidity_data]

        # Prepare the response
        response_dict = {
            "mac_address": mac_address,
            "timestamp": timestamps,
            "temperature": temperatures,
            "humidity": humidities,
        }

        # Return the JSON response
        return json.dumps(response_dict)
        
        response = json.dumps(response_dict)

        return response


## Setup cherrypy and Map objects to their target endpoints

In [ ]:
if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(Sensors(), '/sensors', conf)
    cherrypy.tree.mount(Sensor(), '/sensor', conf)
    cherrypy.tree.mount(HistoricalData(), '/data', conf)  # New endpoint
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5f0c18e4-6e8e-4b7e-be1d-d8fab55c80d7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>